##Multi-Modal Predictive Modeling: Voice, Facial & Product Recommendation Systems
This notebook presents the development of three machine learning models :

**Facial Recognition Model** . Uses engineered image features to identify customers based on facial attributes.

**Voiceprint Verification Model** . Verifies users through voice embedding features.

**Product Recommendation Model** . Predicts product categories based on transaction and engagement data using supervised learning.
All models are evaluated using standard metrics including Accuracy, F1-Score, and Loss etc.

##The Facial Recognition Model


In [2]:
#Import the necessary Python libraries for Facial Recognition
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, log_loss
import joblib

# Load the image features dataset into the notebook
df = pd.read_csv('image_features.csv')

# Display the first five rows
print(df.head())

# Display column information
print(df.info())

  member expression augmentation        feat_0    feat_1    feat_2    feat_3  \
0  david    neutral     original  3.908344e-06  0.000261  0.001713  0.003812   
1  david    neutral      rotated  3.908344e-06  0.000261  0.001713  0.003812   
2  david    neutral      flipped  3.908344e-06  0.000261  0.001713  0.003812   
3  david    neutral    grayscale  1.395837e-07  0.000083  0.002578  0.008346   
4  david      smile     original  4.299178e-05  0.000831  0.002449  0.008304   

     feat_4    feat_5    feat_6  ...   feat_86   feat_87   feat_88   feat_89  \
0  0.015205  0.027483  0.018210  ...  0.001355  0.001092  0.000992  0.000874   
1  0.015205  0.027483  0.018210  ...  0.001355  0.001092  0.000992  0.000874   
2  0.015205  0.027483  0.018210  ...  0.001355  0.001092  0.000992  0.000874   
3  0.024398  0.074433  0.068416  ...       NaN       NaN       NaN       NaN   
4  0.027262  0.021193  0.010250  ...  0.001102  0.001067  0.000954  0.000813   

    feat_90   feat_91   feat_92   feat

In [3]:
# Fill missing values in feature columns with 0.0
feature_cols = [col for col in df.columns if col.startswith('feat_')]
df[feature_cols] = df[feature_cols].fillna(0.0)

# Define features (X) and target (y)
X = df[feature_cols]
y = df['member']

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a RandomForestClassifier model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)


In [5]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
loss = log_loss(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score (macro): {f1:.4f}")
print(f"Log Loss: {loss:.4f}")


Accuracy: 1.0000
F1 Score (macro): 1.0000
Log Loss: 0.0971


In [6]:
#Save the model 'facial_recognition_model.joblib)
joblib.dump(model, 'facial_recognition_model.joblib')

print(" The facial recognition model saved as 'facial_recognition_model.joblib'")

 The facial recognition model saved as 'facial_recognition_model.joblib'


##Voice print Recognition model

In [7]:
# Load the audio features dataset for the voice print recog model
df_audio = pd.read_csv('audio_features.csv')

# View the first few rows
print("Audio Data Head:")
print(df_audio.head())

# View  column information
print("\nAudio Data Info:")
print(df_audio.info())

Audio Data Head:
  member   sample augmentation    energy       rolloff      mfcc_0  \
0  david  approve     original  0.001125   5532.897534 -470.388733   
1  david  approve        pitch  0.000563   6390.997024 -496.560272   
2  david  approve      stretch  0.000520   5738.483481 -502.090332   
3  david  approve        noise  0.001149  18446.853741 -315.086473   
4  david  confirm     original  0.001037   3318.497475 -451.448822   

       mfcc_1     mfcc_2     mfcc_3    mfcc_4     mfcc_5    mfcc_6    mfcc_7  \
0  115.434631  10.793375  36.576359  5.174695  19.042986  2.409958  6.798976   
1  107.548767  14.688319  32.677586  4.130080  18.449661 -0.713317  9.971756   
2  112.294426  10.916822  36.193268  4.142584  18.785419  2.008073  6.771557   
3   26.225565  15.723828  18.246023  7.690150  10.177798  6.418136  4.844470   
4  155.922226  17.962776  33.061432  4.993365  15.411249  0.774826  5.132216   

     mfcc_8    mfcc_9   mfcc_10   mfcc_11   mfcc_12  
0  7.123512  0.306631  5.24

In [8]:
#Select feature columns(assumption: 'energy', 'rolloff', and 'mfcc_' are the feature columns)
#Exclude 'member', 'segmentation' and 'sample' columns
audio_feature_cols = [col for col in df_audio.columns if col not in ['member', 'sample', 'augmentation']]

In [9]:
#Fill any missing values if found ( based on .info() results)
df_audio[audio_feature_cols] = df_audio[audio_feature_cols].fillna(0.0)


In [10]:
#define the features(X)  and target(Y) variables
X_audio = df_audio[audio_feature_cols]
y_audio = df_audio['member']


In [11]:
# Split the data into training and testing sets
X_audio_train, X_audio_test, y_audio_train, y_audio_test = train_test_split(
    X_audio, y_audio, test_size=0.3, random_state=42, stratify=y_audio
)



In [12]:
# Train a RandomForestClassifier model for voiceprint verification
voice_model = RandomForestClassifier(random_state=42)
voice_model.fit(X_audio_train, y_audio_train)

# Make predictions
y_audio_pred = voice_model.predict(X_audio_test)
y_audio_pred_proba = voice_model.predict_proba(X_audio_test)


In [13]:
# Evaluate the model
audio_accuracy = accuracy_score(y_audio_test, y_audio_pred)
audio_f1 = f1_score(y_audio_test, y_audio_pred, average='macro')
audio_loss = log_loss(y_audio_test, y_audio_pred_proba)

print(f"\nVoiceprint Verification Model Performance:")
print(f"Accuracy: {audio_accuracy:.4f}")
print(f"F1 Score (macro): {audio_f1:.4f}")
print(f"Log Loss: {audio_loss:.4f}")

# Save the trained voiceprint verification model
joblib.dump(voice_model, 'voiceprint_verification_model.joblib')
print("\nVoiceprint verification model saved as 'voiceprint_verification_model.joblib'")



Voiceprint Verification Model Performance:
Accuracy: 1.0000
F1 Score (macro): 1.0000
Log Loss: 0.3884

Voiceprint verification model saved as 'voiceprint_verification_model.joblib'
